In [15]:
import pandas as pd
import folium
import psycopg2
from dotenv import load_dotenv
import folium
import os


In [17]:
load_dotenv('credentials.env')

# Print out the environment variables to confirm they're loaded
print("DB_HOST:", os.getenv("DB_HOST"))
print("DB_NAME:", os.getenv("DB_NAME"))
print("DB_USER:", os.getenv("DB_USER"))
print("DB_PASSWORD:", os.getenv("DB_PASSWORD"))
print("DB_PORT:", os.getenv("DB_PORT"))



DB_HOST: pinot.cs.ucsb.edu
DB_NAME: pinot
DB_USER: cs190nf24g10
DB_PASSWORD: YKUkr3GV
DB_PORT: 5432


In [21]:
try:
    conn = psycopg2.connect(
        host=os.getenv("DB_HOST"),
        database=os.getenv("DB_NAME"),
        user=os.getenv("DB_USER"),
        password=os.getenv("DB_PASSWORD"),
        port=os.getenv("DB_PORT")
    )
    print("Connected successfully.")
except Exception as e:
    print("Failed to connect to the database.")
    print("Error:", e)

Connected successfully.


In [22]:
try:
    if conn is not None:
        cur = conn.cursor()
        query = "SELECT label, time, signal, packet_loss FROM public.wifistats LIMIT 100;"
        cur.execute(query)
        data = cur.fetchall()
        for row in data:
            print(row)
        cur.close()
    else:
        print("Connection was not established.")
except Exception as e:
    print("Error executing the query.")
    print("Error:", e)
finally:
    # Check if the connection is open before trying to close it
    if conn is not None and conn.closed == 0:
        conn.close()
        print("Database connection closed.")

('raspi-e4:5f:01:ac:ed:5b', datetime.datetime(2024, 6, 24, 2, 39, 40, 280950), -66.0, 0.0)
('raspi-e4:5f:01:8d:ca:34', datetime.datetime(2024, 6, 24, 2, 39, 40, 281437), -65.0, 0.0)
('raspi-e4:5f:01:9c:20:81', datetime.datetime(2024, 6, 24, 2, 39, 40, 281833), -66.0, 0.0)
('raspi-e4:5f:01:8d:07:fe', datetime.datetime(2024, 6, 24, 2, 39, 40, 282211), -75.0, 0.0)
('raspi-e4:5f:01:8c:8a:41', datetime.datetime(2024, 6, 24, 2, 49, 55, 886456), -51.0, 0.0)
('raspi-e4:5f:01:56:d8:f3', datetime.datetime(2024, 6, 24, 2, 49, 55, 888764), -69.0, 0.0)
('raspi-e4:5f:01:9c:24:87', datetime.datetime(2024, 6, 24, 2, 49, 55, 889293), -53.0, 0.0)
('raspi-e4:5f:01:72:a4:93', datetime.datetime(2024, 6, 24, 2, 49, 55, 889729), -55.0, 0.0)
('raspi-e4:5f:01:6c:98:83', datetime.datetime(2024, 6, 24, 2, 49, 55, 890146), -46.0, 0.0)
('raspi-e4:5f:01:8b:bc:d9', datetime.datetime(2024, 6, 24, 2, 49, 55, 890686), -58.0, 0.0)
('raspi-e4:5f:01:a0:4a:dd', datetime.datetime(2024, 6, 24, 2, 49, 55, 891140), -40.0, 0.0)